# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/GitHub_Repositories/API_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
## import csv to dataframe, add try block to deal with directory issues
csv_import_path = os.path.join('..','Weather','Export Folder','city_data.csv')
city_weather_df=pd.read_csv(csv_import_path)

city_weather_df.head()

# # ### using this to limit the request, remove once completed
# city_weather_df=city_weather_df.head()
# city_weather_df.head(20)

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,54.98,-117.91,valleyview,ca,"(55.066667, -117.283333)",55.066667,-117.283333,30.88,95,99,7.36
1,53.60,-13.37,dingle,ie,"(52.1408333, -10.2688889)",52.140833,-10.268889,52.36,83,100,10.71
2,-32.08,-16.69,jamestown,sh,"(-15.9333333, -5.7166667)",-15.933333,-5.716667,62.64,89,100,23.49
3,37.95,99.76,zhangye,cn,"(38.934167, 100.451667)",38.934167,100.451667,46.98,48,4,16.04
4,49.51,140.54,vanino,ru,"(49.086856, 140.254337)",49.086856,140.254337,44.83,64,24,6.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
#configure gmaps with key
gmaps.configure(api_key=g_key)

In [45]:

#defines coordinates to plot and weight is humididty
### shoul these be .astype(float)
locations=city_weather_df[['Latitude','Longitude']]
weighty=city_weather_df['Humidity']

fig=gmaps.figure(center=(0,0),zoom_level=2)
heat=gmaps.heatmap_layer(locations,weights=weighty,dissipating=True,max_intensity=100,point_radius=10)
fig.add_layer(heat)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
#narrows down the dataframe to a dataframe with ideal weather
ideal_weather = city_weather_df.loc[(city_weather_df['Temperature']>70)&(city_weather_df['Temperature']<80)&(city_weather_df['Wind Speed']<10)&(city_weather_df['Cloudiness']==0),:]

#drops bad data
ideal_weather.dropna()


ideal_weather.head()

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
60,-52.57,48.58,tsihombe,mg,"(-25.3, 45.4833333)",-25.300000,45.483333,72.05,66,0,9.95
81,27.42,79.51,farrukhabad,in,"(27.4, 79.566667)",27.400000,79.566667,79.43,26,0,6.29
134,23.51,1.47,tessalit,ml,"(20.2013889, 1.0088889)",20.201389,1.008889,72.54,20,0,7.11
157,31.12,81.91,dharchula,in,"(29.85, 80.533333)",29.850000,80.533333,71.13,20,0,3.74
216,13.23,58.07,salalah,om,"(17.0175, 54.0827778)",17.017500,54.082778,75.20,78,0,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
#simplifies and defines the hotel_df
hotel_df=ideal_weather.drop(['Approximate Lat','Approximate Lon','Exact Coordinates'],axis=1).reset_index(drop=True)

hotel_df.head()
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lon']=""
hotel_df=hotel_df.rename(columns={'City Name':'City','Country Code':'Country'})
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon
0,tsihombe,mg,-25.300000,45.483333,72.05,66,0,9.95,,,
1,farrukhabad,in,27.400000,79.566667,79.43,26,0,6.29,,,
2,tessalit,ml,20.201389,1.008889,72.54,20,0,7.11,,,
3,dharchula,in,29.850000,80.533333,71.13,20,0,3.74,,,
4,salalah,om,17.017500,54.082778,75.20,78,0,2.24,,,


In [48]:

### finds the closest hotel and fills in the hotel name and coordinates to the dataframe
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

for i,r in hotel_df.iterrows():
    c_lat=hotel_df['Latitude'][i]
    c_lon = hotel_df['Longitude'][i]
    target_coordinates =f'{c_lat}, {c_lon}' 
    
    param = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    places_json=requests.get(base_url,params=param).json()
    ## tessalit is literally in the middle of the sahara desert and returns no results, so this is a work around

    if places_json['status']=='ZERO_RESULTS':
        hotel_df['Hotel Name'][i]=np.nan
    else:
        hotel_df['Hotel Name'][i]=places_json["results"][0]['name']
        hotel_df['Hotel Lat'][i]=places_json["results"][0]['geometry']['location']['lat']
        hotel_df['Hotel Lon'][i]=places_json["results"][0]['geometry']['location']['lng']


## Dropping the cities that returned no results 
hotel_df=hotel_df.dropna()

hotel_df.head(30)


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon
0,tsihombe,mg,-25.300000,45.483333,72.05,66,0,9.95,Restaurant Le Paradis du sud,-25.3187,45.4837
1,farrukhabad,in,27.400000,79.566667,79.43,26,0,6.29,Hotel Mohan Palace,27.385,79.5742
3,dharchula,in,29.850000,80.533333,71.13,20,0,3.74,Hotel new yash dharchula,29.8489,80.5415
4,salalah,om,17.017500,54.082778,75.20,78,0,2.24,HYATT SALALAH HOTEL,17.0194,54.1108
5,yulara,au,-25.245291,130.980545,78.80,16,0,9.17,Ayers Rock Resort,-25.2377,130.986
6,tehri,in,30.383333,78.483333,75.22,20,0,1.23,Tehri Himalayan Residency,30.383,78.4394
7,mangochi,mw,-14.466667,35.266667,76.28,57,0,3.00,Sunbird Nkopola Lodge,-14.4862,35.2533
8,mecca,sa,21.426667,39.826111,75.72,47,0,1.23,Pullman Zamzam Makkah,21.4193,39.8247
9,jizan,sa,16.889167,42.551111,78.80,78,0,3.36,Radisson Blu Resort Jizan,16.9292,42.5485
11,elat,il,29.558049,34.948212,71.60,68,0,9.17,"Al Manara, a Luxury Collection Hotel, Saraya A...",29.5358,34.9933


In [49]:
# # #create a new folder and exports the dataframe

def dataframe_to_csv(df):
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export Folder')
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    csv_export_path = os.path.join(new_folder_path,'hotel_data.csv')
    df.to_csv(csv_export_path,index=False)
dataframe_to_csv(hotel_df)

In [50]:
## import the csv file you just saved
def import_path():
    current_directory=os.getcwd()
    import_path.new_folder_path =os.path.join(current_directory,r'Export Folder')
    csv_export_path = os.path.join(import_path.new_folder_path,'hotel_data.csv')
    return csv_export_path

im_path=import_path()
csv_imp_df=pd.read_csv(im_path,delimiter=',')
csv_imp_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Lat,Hotel Lon
0,tsihombe,mg,-25.300000,45.483333,72.05,66,0,9.95,Restaurant Le Paradis du sud,-25.318744,45.483670
1,farrukhabad,in,27.400000,79.566667,79.43,26,0,6.29,Hotel Mohan Palace,27.385038,79.574159
2,dharchula,in,29.850000,80.533333,71.13,20,0,3.74,Hotel new yash dharchula,29.848928,80.541453
3,salalah,om,17.017500,54.082778,75.20,78,0,2.24,HYATT SALALAH HOTEL,17.019380,54.110750
4,yulara,au,-25.245291,130.980545,78.80,16,0,9.17,Ayers Rock Resort,-25.237719,130.985966


In [51]:
### plotting the hotel latitude and longitude
y_locations=city_weather_df[['Latitude','Longitude']]
x_locations=csv_imp_df[['Hotel Lat','Hotel Lon']]
x_weighty=city_weather_df['Humidity']



x_fig=gmaps.figure(center=(0,0),zoom_level=2)
x_heat=gmaps.heatmap_layer(y_locations,weights=x_weighty,dissipating=True,max_intensity=100,point_radius=10)
x_marks=gmaps.marker_layer(x_locations,info_box_content="filler info box text")
x_fig.add_layer(x_marks)
x_fig.add_layer(x_heat)

x_fig

Figure(layout=FigureLayout(height='420px'))

In [52]:
## not sure what do do with this code

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# x_locations = hotel_df[["Latitude", "Longitude"]]

# hotel_df["Hotel Info"]=hotel_info

# hotel_df.head()

In [53]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [54]:
# Add marker layer ontop of heat map


# Display figure
